In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import argparse
import pandas as pd
import glob
import os
import cv2
import random as rand
import os
import xml.etree.ElementTree as et
import re
import pandas as pd

In [2]:
bases_prontas_path = os.path.join("D:\\","FIA","TCC","BASES","")
print(bases_prontas_path)

D:\FIA\TCC\BASES\


## Criar modelos com transfer learning

In [ ]:
baseModel_ResNet50V2_64 = ResNet50V2(weights="imagenet", 
                                     include_top=False,
                                     input_tensor=Input(shape=(64, 64, 3)))

In [ ]:
# construct the head of the model that will be placed on top of the
# the base model
headModel_ResNet50V2_64 = baseModel_ResNet50V2_64.output
# headModel_ResNet50V2_64 = AveragePooling2D(pool_size=(7, 7))(headModel_ResNet50V2_64)
headModel_ResNet50V2_64 = AveragePooling2D(pool_size=(2, 2))(headModel_ResNet50V2_64)
headModel_ResNet50V2_64 = Flatten(name="flatten")(headModel_ResNet50V2_64)
headModel_ResNet50V2_64 = Dense(12288, activation="relu")(headModel_ResNet50V2_64)
headModel_ResNet50V2_64 = Dropout(0.3)(headModel_MobileNetV2_224)
headModel_ResNet50V2_64 = Dense(4096, activation="relu")(headModel_ResNet50V2_64)
headModel_ResNet50V2_64 = Dropout(0.3)(headModel_MobileNetV2_224)
headModel_ResNet50V2_64 = Dense(3, activation="softmax")(headModel_ResNet50V2_64)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model_based_ResNet50V2_64 = Model(inputs=baseModel_ResNet50V2_64.input, 
                                  outputs=headModel_ResNet50V2_64)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel_ResNet50V2_64.layers:
    layer.trainable = False

In [ ]:
baseModel_ResNet50V2_224 = ResNet50V2(weights="imagenet", 
                                      include_top=False,
                                      input_tensor=Input(shape=(224, 224, 3)))

In [ ]:
# construct the head of the model that will be placed on top of the
# the base model
headModel_ResNet50V2_224 = baseModel_ResNet50V2_224.output
headModel_ResNet50V2_224 = AveragePooling2D(pool_size=(7, 7))(headModel_ResNet50V2_224)
# headModel_ResNet50V2_224 = AveragePooling2D(pool_size=(2, 2))(headModel_ResNet50V2_224)
headModel_ResNet50V2_224 = Flatten(name="flatten")(headModel_ResNet50V2_224)
headModel_ResNet50V2_224 = Dense(12288, activation="relu")(headModel_ResNet50V2_224)
headModel_ResNet50V2_224 = Dropout(0.3)(headModel_MobileNetV2_224)
headModel_ResNet50V2_224 = Dense(4096, activation="relu")(headModel_ResNet50V2_224)
headModel_ResNet50V2_224 = Dropout(0.3)(headModel_MobileNetV2_224)
headModel_ResNet50V2_224 = Dense(3, activation="softmax")(headModel_ResNet50V2_224)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model_based_ResNet50V2_224 = Model(inputs=baseModel_ResNet50V2_224.input, 
                                   outputs=headModel_ResNet50V2_224)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel_ResNet50V2_224.layers:
    layer.trainable = False

In [6]:
INIT_LR = 1e-4
EPOCHS = 100
BS = 1

aug = ImageDataGenerator(zoom_range=0.1,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.15,
                         fill_mode="nearest")

In [8]:
print("[INFO] compiling model...")
opt1 = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
# opt2 = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

[INFO] compiling model...


## Compilar e treinar os modelos criados

In [ ]:
dataset = np.load(bases_prontas_path+'mask_dataset_vgg16_preprocess_input_64_64_3.npy')
target = np.load(bases_prontas_path+'mask_dataset_labels.npy')
(trainX, testX, trainY, testY) = train_test_split(dataset, 
                                                  target,
                                                  test_size=0.3, 
                                                  stratify=target, 
                                                  random_state=42)

In [ ]:
model_based_ResNet50V2_64.compile(loss="categorical_crossentropy", 
                                  optimizer=opt1,
                                  metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H_ResNet50V2_64 = model_based_ResNet50V2_64.fit(aug.flow(trainX, trainY, batch_size=BS),
                                                steps_per_epoch=len(trainX) // BS,
                                                validation_data=(testX, testY),
                                                validation_steps=len(testX) // BS,
                                                epochs=EPOCHS)


## Avaliar os modelos

In [20]:
print("[INFO] evaluating network...")
lb = ["without_mask","mask_weared_incorrect","with_mask"]
def Avaliando_modelo(model, NWHead = None,x_test=testX, y_test=testY):
    predIdxs = model.predict(x_test, batch_size=32)

    # for each image in the testing set we need to find the index of the
    # label with corresponding largest predicted probability
    predIdxs = np.argmax(predIdxs, axis=1)

    # show a nicely formatted classification report
    print(classification_report(y_test.argmax(axis=1), predIdxs,
                                target_names=lb))

    # serialize the model to disk
    print("[INFO] saving mask detector model...")

    if NWHead:
        # plot the training loss and accuracy
        N = EPOCHS
        plt.style.use("ggplot")
        plt.figure()
        plt.plot(np.arange(0, N), NWHead.history["loss"], label="train_loss")
        plt.plot(np.arange(0, N), NWHead.history["val_loss"], label="val_loss")
        plt.plot(np.arange(0, N), NWHead.history["accuracy"], label="train_acc")
        plt.plot(np.arange(0, N), NWHead.history["val_accuracy"], label="val_acc")
        plt.title("Training Loss and Accuracy")
        plt.xlabel("Epoch #")
        plt.ylabel("Loss/Accuracy")
        plt.legend(loc="lower left")
        plt.show()
    return classification_report(y_test.argmax(axis=1), predIdxs,
                                target_names=lb,output_dict=True)

[INFO] evaluating network...


In [ ]:
Avaliando_modelo(model_based_ResNet50V2_64, H_ResNet50V2_64, testX, testY)

## salvar os pesos dos modelos para não precisar salvar de novo

In [ ]:
model_path = os.path.join("D:\\","FIA","TCC","Meus_Modelos","")
print(model_path)

In [ ]:
# https://www.tensorflow.org/tutorials/keras/save_and_load
# Save the weights
model_based_ResNet50V2_64.save_weights(model_path+'model_based_ResNet50V2_64_checkpoint')


## Refazer para um dataset com shape maior

In [ ]:
dataset = np.load(bases_prontas_path+'mask_dataset_vgg16_preprocess_input_224_224_3.npy')
target = np.load(bases_prontas_path+'mask_dataset_labels.npy')
(trainX, testX, trainY, testY) = train_test_split(dataset, 
                                                  target,
                                                  test_size=0.3, 
                                                  stratify=target, 
                                                  random_state=42)

In [ ]:
model_based_ResNet50V2_224.compile(loss="categorical_crossentropy", 
                                   optimizer=opt1,
                                   metrics=["accuracy"])

# train the head of the network
print("[INFO] training head...")
H_ResNet50V2_224 = model_based_ResNet50V2_224.fit(aug.flow(trainX, trainY, batch_size=BS),
                                                  steps_per_epoch=len(trainX) // BS,
                                                  validation_data=(testX, testY),
                                                  validation_steps=len(testX) // BS,
                                                  epochs=EPOCHS)


In [ ]:
Avaliando_modelo(model_based_ResNet50V2_224, H_ResNet50V2_224, testX, testY)

In [ ]:
model_based_ResNet50V2_224.save_weights(model_path+'model_based_ResNet50V2_224_checkpoint')